# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D_FirstPeriodStudy");

Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
//var myBatch = ExecutionQueues[0];
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


A switch for running the worksheet locally (since we have a folder structure for the data, which is not copied on Jenkins for testing)

In [ ]:
bool NunitTest = false;

In [ ]:
//foreach(var s in wmg.Sessions)
//    s.Delete(true);

## Verification of Initial Value data

Initial values and parameters for the simulation (intial droplet shape, Ohnesorg number, initial velocity)
were specified by project partner (TU Graz, Group Prof. Brenn).
Details can be found in Document `setup.pdf`, to be found in the same directory as this worksheet.

First, it is verified that the initial values chosen here actually match the specification.

In [ ]:
//MultidimensionalArray[] ReferenceData = new MultidimensionalArray[5];
int[] modes = new int[] { 3 };
List<string> cases = new List<string>();
int nCase = 0;
// add simulations for mode 2
if(modes.Contains(2)) {
    cases.Add("m2_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta03");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 3
if(modes.Contains(3)) {
    cases.Add("m3_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 4
if(modes.Contains(4)) {
    cases.Add("m4_Oh01_eta07");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta06");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta05");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta03");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta02");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
int numCases = cases.Count;
numCases

1

In [ ]:
bool[] refDataProvided = new bool[numCases];
refDataProvided[0] = true;
// //refDataProvided[7] = true;
// refDataProvided[10] = true;
// refDataProvided[11] = true;

Load reference data for all cases; These files contain two columns, i.e. the azimuth angle and the respective droplet radius.

In [ ]:
MultidimensionalArray[] ReferenceData = new MultidimensionalArray[numCases];
for(int iCase = 0; iCase < numCases; iCase++) {
    if(refDataProvided[iCase]) {
    if(!NunitTest)  
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/surfaceDrop_{cases[iCase]}.txt");
    else
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"surfaceDrop_{cases[iCase]}.txt");
}
}

Analytical expressions for the reference data (see `setup.pdf`); this is to verify that the definition of Legendre
Functions resp. Polynomials in BoSSS actually matches the definition used by the TU Graz group.

In [ ]:
Dictionary<string, Func<double, double>> casesRadius = new Dictionary<string, Func<double, double>>();

In [ ]:
double caseR_m2eta07(double angle) {
   double radius = 0.895131 + 0.7*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta07", caseR_m2eta07);

In [ ]:
double caseR_m2eta06(double angle) {
   double radius = 0.923706 + 0.6*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta06", caseR_m2eta06);

In [ ]:
double caseR_m2eta05(double angle) {
   double radius = 0.947538 + 0.5*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta05", caseR_m2eta05);

In [ ]:
double caseR_m2eta03(double angle) {
    double radius = 0.981486 + 0.3*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m2_Oh01_eta03", caseR_m2eta03);

In [ ]:
double caseR_m3eta07(double angle) {
    double radius = 0.930122 + 0.7*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta07", caseR_m3eta07);

In [ ]:
double caseR_m3eta06(double angle) {
    double radius = 0.948619 + 0.6*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta06", caseR_m3eta06);

In [ ]:
double caseR_m3eta05(double angle) {
    double radius = 0.964301 + 0.5*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta05", caseR_m3eta05);

In [ ]:
double caseR_m3eta03(double angle) {
    double radius = 0.987144 + 0.3*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta03", caseR_m3eta03);

In [ ]:
double caseR_m4eta07(double angle) {
    double radius = 0.943440 + 0.7*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta07", caseR_m4eta07);

In [ ]:
double caseR_m4eta06(double angle) {
    double radius = 0.958674 + 0.6*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta06", caseR_m4eta06);

In [ ]:
double caseR_m4eta05(double angle) {
    double radius = 0.971459 + 0.5*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta05", caseR_m4eta05);

In [ ]:
double caseR_m4eta03(double angle) {
    double radius = 0.989838 + 0.3*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta03", caseR_m4eta03);

In [ ]:
double caseR_m4eta02(double angle) {
    double radius = 0.995508 + 0.2*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta02", caseR_m4eta02);

Conversion to cartesian coordinates in order to match the data and verification against analytical expression:

In [ ]:
double[][] refX = new double[numCases][];
double[][] refZ = new double[numCases][];
double[][] caseX = new double[numCases][];
double[][] caseZ = new double[numCases][];
for(int iCase = 0; iCase < numCases; iCase++) {

    if(refDataProvided[iCase]) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    double[] radius = ReferenceData[iCase].GetColumn(1);
       
    double RadiusErrorNorm = 0.0;
    int I = angle.Length;
    double[] x1 = new double[I], z1 = new double[I];
    double[] cx1 = new double[I], cz1 = new double[I];
    for(int i = 0; i < I; i++) {
        x1[i] = Math.Sin(angle[i])*radius[i];
        z1[i] = Math.Cos(angle[i])*radius[i];
        
        double radius_expr = casesRadius[cases[iCase]](angle[i]);
        RadiusErrorNorm += (radius[i] - radius_expr).Pow2();     
        
        cx1[i] = Math.Sin(angle[i])*radius_expr;
        cz1[i] = Math.Cos(angle[i])*radius_expr;
    } 
    refX[iCase] = x1;
    refZ[iCase] = z1;
    caseX[iCase] = cx1;
    caseZ[iCase] = cz1;

    RadiusErrorNorm = RadiusErrorNorm.Sqrt();
    Console.WriteLine($"Comparison error for radius in case {iCase + 1}: {RadiusErrorNorm}");
    // Note: since the factors in `setup.pdf` are only provided up to 6 digits, an error threshold of 1e-5 seems reasonable.
    Assert.LessOrEqual(RadiusErrorNorm, 1e-5, "Error in comparing reference data against Legendre polynomials in BoSSS");
}
}

Error: NUnit.Framework.AssertionException:   Error in comparing reference data against Legendre polynomials in BoSSS
  Expected: less than or equal to 1.0000000000000001E-05d
  But was:  0.031625997428843741d

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.LessOrEqual(Double arg1, Double arg2, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Comparisons.cs:line 943
   at Submission#27.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmiss

### Plot of Reference Data

In [ ]:
int refCase = 0;
Plot(refX[refCase], refZ[refCase], "Ref-Case", ". black",
    caseX[refCase], caseZ[refCase], "Case", ". blue")

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 Ref-Case 
 
 
 Ref-Case 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M716.2,34.7 L758.4,34.7 M61.6,63.9 L69.3,64.0 L77.0,64.2 L84.7,64.4 L92.4,64.7 L100.1,65.1
 L107.7,65.5 L115.3,66.0 L122.9,66.6 L130.4,67.2 L137.9,67.9 L145.4,68.7 L152.8,69.5 L160.2,70.4
 L167.5,71.3 L174.7,72.3 L181.9,73.4 L189.0,74.5 L196.1,75.7 L203.1,77.0 L210.0,78.3 L216.9,79.6
 L223.6,81.0 L230.3,82.5 L236.9,84.0 L243.4,85.6 L249.8,87.2 L256.2,88.8 L262.4,90.5 L268.6,92.3
 L274.6,94.1 L280.6,95.9 L286.4,97.8 L292.2,99.7 L297.8,101.6 L303.4,103.6 L308.8,105.7 L314.1,107.7
 L319.4,109.8 L324.5,111.9 L329.5,114.1 L334.4,116.2 L339.2,118.4 L343.9,120.6 L348.5,122.9 L353.0,125.1
 L357.4,127.4 L361.7,129.7 L365.8,132.0 L369.9,134.3 L373.9,136.7 L377.7,139.0 L381.5,141.4 L385.2,143.7
 L388.7,146.1 L392.2,148.5 L395.6,150.9 L398.9,153.3 L402.1,155.6 L405.2,158.0 L408.2,160.4 L411.1,162.8
 L414.0,165.1 L416.8,167.5 L419.5,169.9 L422.1,172.2 L424.7,174.6 L427.2,176.9 L429.6,179.2 L432.0,181.6
 L434.3,183.9 L436.5,186.2 L438.7,188.4 L440.9,190.7 L443.0,192.9 L445.0,195.2 L447.0,197.4 L449.0,199.6
 L450.9,201.8 L452.9,203.9 L454.7,206.1 L456.6,208.2 L458.4,210.3 L460.3,212.4 L462.1,214.5 L463.8,216.5
 L465.6,218.5 L467.4,220.5 L469.2,222.5 L470.9,224.5 L472.7,226.5 L474.5,228.4 L476.3,230.3 L478.1,232.2
 L479.9,234.1 L481.7,236.0 L483.6,237.8 L485.4,239.6 L487.3,241.4 L489.2,243.2 L491.2,245.0 L493.1,246.8
 L495.1,248.5 L497.2,250.3 L499.2,252.0 L501.3,253.7 L503.5,255.4 L505.6,257.1 L507.9,258.8 L510.1,260.5
 L512.4,262.2 L514.7,263.8 L517.1,265.5 L519.5,267.1 L522.0,268.8 L524.5,270.4 L527.0,272.0 L529.6,273.7
 L532.3,275.3 L534.9,276.9 L537.6,278.6 L540.4,280.2 L543.2,281.9 L546.0,283.5 L548.9,285.2 L551.8,286.8
 L554.7,288.5 L557.7,290.2 L560.7,291.8 L563.7,293.5 L566.8,295.2 L569.8,296.9 L572.9,298.6 L576.1,300.4
 L579.2,302.1 L582.4,303.9 L585.5,305.7 L588.7,307.4 L591.9,309.3 L595.1,311.1 L598.3,312.9 L601.5,314.8
 L604.7,316.6 L607.9,318.5 L611.0,320.4 L614.2,322.4 L617.3,324.3 L620.4,326.3 L623.5,328.3 L626.6,330.3
 L629.6,332.3 L632.6,334.4 L635.5,336.4 L638.5,338.5 L641.3,340.6 L644.1,342.8 L646.9,344.9 L649.6,347.1
 L652.2,349.3 L654.8,351.5 L657.3,353.7 L659.7,355.9 L662.0,358.2 L664.3,360.5 L666.5,362.8 L668.5,365.1
 L670.5,367.4 L672.4,369.8 L674.2,372.1 L675.9,374.5 L677.5,376.9 L679.0,379.3 L680.4,381.7 L681.6,384.1
 L682.8,386.6 L683.8,389.0 L684.7,391.5 L685.5,393.9 L686.1,396.4 L686.6,398.9 L687.0,401.3 L687.2,403.8
 L687.3,406.3 L687.3,408.7 L687.1,411.2 L686.8,413.7 L686.3,416.1 L685.7,418.6 L684.9,421.1 L684.0,423.5
 L683.0,425.9 L681.8,428.4 L680.4,430.8 L678.9,433.2 L677.2,435.6 L675.4,437.9 L673.4,440.3 L671.3,442.6
 L669.1,444.9 L666.7,447.2 L664.1,449.5 L661.4,451.8 L658.5,454.0 L655.5,456.2 L652.3,458.4 L649.0,460.5
 L645.6,462.6 L642.0,464.7 L638.3,466.7 L634.4,468.8 L630.4,470.7 L626.3,472.7 L622.0,474.6 L617.6,476.5
 L613.1,478.3 L608.4,480.1 L603.6,481.9 L598.7,483.6 L593.7,485.2 L588.6,486.9 L583.4,488.5 L578.0,490.0
 L572.6,491.5 L567.1,493.0 L561.4,494.4 L555.7,495.7 L549.9,497.0 L544.0,498.3 L538.0,499.5 L531.9,500.7
 L525.8,501.8 L519.6,502.9 L513.3,503.9 L507.0,504.9 L500.6,505.8 L494.2,506.7 L487.7,507.5 L481.2,508.3
 L474.6,509.1 L468.0,509.8 L461.3,510.4 L454.6,511.0 L447.9,511.6 L441.2,512.1 L434.5,512.5 L427.7,512.9
 L420.9,513.3 L414.2,513.6 L407.4,513.9 L400.6,514.2 L393.8,514.4 L387.1,514.5 L380.3,514.6 L373.6,514.7
 L366.8,514.8 L360.2,514.8 L353.5,514.8 L346.8,514.7 L340.2,514.6 L333.6,514.5 L327.1,514.3 L320.6,514.1
 L314.1,513.9 L307.7,513.7 L301.3,513.4 L295.0,513.1 L28

### Matching of the Spherical Harmonics against the provided Data

In [ ]:
Dictionary<string, Formula> casesPhi = new Dictionary<string, Formula>();

In [ ]:
var Phi_m2eta07_Init = new Formula(
"Phi1",
false,
"using ilPSP.Utils; " + 
"double Phi1(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.895131*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"}");
casesPhi.Add("m2_Oh01_eta07", Phi_m2eta07_Init);

In [ ]:
var Phi_m2eta06_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.923706*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta06", Phi_m2eta06_Init);

In [ ]:
var Phi_m2eta05_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.947538*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta05", Phi_m2eta05_Init);

In [ ]:
var Phi_m2eta03_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.981486*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta03", Phi_m2eta03_Init);

In [ ]:
var Phi_m3eta07_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.930122*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta07", Phi_m3eta07_Init);

In [ ]:
var Phi_m3eta06_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.948619*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta06", Phi_m3eta06_Init);

In [ ]:
var Phi_m3eta05_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.964301*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta05", Phi_m3eta05_Init);

In [ ]:
var Phi_m3eta03_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.987144*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta03", Phi_m3eta03_Init);

In [ ]:
var Phi_m4eta07_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.943440*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.7*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta07", Phi_m4eta07_Init);

In [ ]:
var Phi_m4eta06_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.958674*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.6*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta06", Phi_m4eta06_Init);

In [ ]:
var Phi_m4eta05_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.971459*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.5*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta05", Phi_m4eta05_Init);

In [ ]:
var Phi_m4eta03_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.989838*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.3*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta03", Phi_m4eta03_Init);

In [ ]:
var Phi_m4eta02_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.995508*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.2*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta02", Phi_m4eta02_Init);

In [ ]:
// IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init, Phi2Init, Phi3Init, Phi4Init, Phi5Init};

In [ ]:
for(int iCase = 0; iCase < numCases; iCase++) {
    if(refDataProvided[iCase]) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    //double[] xI = refX[iCase];   
    //double[] zI = refZ[iCase];
    int I = angle.Length;
    
    double PhiErr = 0;
    for(int i = 0; i < I; i++) {
        double radius_expr = casesRadius[cases[iCase]](angle[i]);    
        double x1 = Math.Sin(angle[i])*radius_expr;
        double z1 = Math.Cos(angle[i])*radius_expr;
    
        PhiErr += casesPhi[cases[iCase]].Evaluate(new Vector(x1, 0, z1), 0.0).Abs();
    }
    Console.WriteLine($"Phi error for case {iCase}: {PhiErr}");
    Assert.LessOrEqual(PhiErr, 1e-10, "Level-Set function is not zero at desired surface.");
    
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e-5, 1e-5, 1e-5), 0.0) < 0, "Inside must be phase A/negative");
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e+1, 1e+1, 1e+1), 0.0) > 0, "Outside must be phase B/positive");
}
}

### Initial Velocities

In [ ]:
// //var anaVel = new IBoundaryAndInitialData[numCases,3];
// Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)> analyticVel = new Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)>();

// for(int iCase = 0; iCase < numCases; iCase++) {
//     MultidimensionalArray polVel;
//     MultidimensionalArray radVel;
//     if (!NunitTest) {
//         polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/polarVel_{cases[iCase]}.txt");
//         radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/radialVel_{cases[iCase]}.txt");
//     } else {
//         polVel = IMatrixExtensions.LoadFromTextFile($"polarVel_{cases[iCase]}.txt");
//         radVel = IMatrixExtensions.LoadFromTextFile($"radialVel_{cases[iCase]}.txt");
//     }
//     Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
//     Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));
    
//     double[] radiusS = polVel.GetColumn(0);
//     double[] anglesS = polVel.GetColumn(1);
//     double[] polVelS = polVel.GetColumn(2);
//     double[] radVelS = radVel.GetColumn(2);
    
//     var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
//     velX.SetData(anglesS, radiusS, polVelS, radVelS);
//     var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
//     velY.SetData(anglesS, radiusS, polVelS, radVelS);
//     var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
//     velZ.SetData(anglesS, radiusS, polVelS, radVelS);
    
//     // anaVel[iCase, 0] = velX;
//     // anaVel[iCase, 1] = velY;
//     // anaVel[iCase, 2] = velZ;
//     analyticVel.Add(cases[iCase], (velX, velY, velZ));
// }

## Grid Creation

In [ ]:
//foreach(var g in BoSSSshell.WorkflowMgm.Grids)
//   g.Delete(true);

### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 7 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_wallBC_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

In [ ]:
gridTypes

key,value
wallBC,[ { Guid = 97d33818-3ae1-4cae-b351-690d789c73c1; Name = OscillatingDroplet3D_7x7x14_wallBC_quarterDomain; Cell Count = 686; Dim = 3 } ]


In [ ]:
//var g = (wmg.Grids[0] as GridProxy).RealGrid;

In [ ]:
//(g.iGridData as GridData).GlobalBoundingBox

In [ ]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for all solver runs

In [ ]:
cases

index,value
0,m3_Oh01_eta03


In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta07", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta06", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta05", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta03", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m3_Oh01_eta07", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta06", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta05", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta03", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta07", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta06", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta05", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta03", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta02", (0.1, 1, 5e-3, 300));

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC" }; ///{ "wallBC", "pressureOutletBC", "refinedTest" };

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool[] useInitial = new bool[] { false };
bool[] useNewton = new bool[] { false };

// string grdKey = gridTypeKeys[1];
// Grids = gridTypes[grdKey];

foreach(bool bInitial in useInitial) {
foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(string grdKey in gridTypeKeys) {
foreach(var grd in gridTypes[grdKey]) {
int iCase = 0;
foreach(var myCase in cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = casesSetUp[myCase].AMRlevel;
    string JobName = $"OD3D_J{J}k{k}_{grdKey}_amr{AMRlvl}_{myCase}";
    if(bInitial) {
        JobName = JobName + "_thirdOrderInit";
    }
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    C.InitialValues.Add("Phi", casesPhi[myCase]);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
        C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    }
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase].dt;
    C.NoOfTimesteps = casesSetUp[myCase].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    // if(bInitial) {
    //     C.AddInitialValue("VelocityX#A", analyticVel[myCase].velX);
    //     //C.AddInitialValue("VelocityX#B", anaVel[myCase.Case - 1, 0]);
    //     C.AddInitialValue("VelocityY#A", analyticVel[myCase].velY);
    //     //C.AddInitialValue("VelocityY#B", anaVel[myCase.Case - 1, 1]);
    //     C.AddInitialValue("VelocityZ#A", analyticVel[myCase].velZ);
    //     //C.AddInitialValue("VelocityZ#B", anaVel[myCase.Case - 1, 2]);
    // }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}
}
}

In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03


In [ ]:
// foreach(var ctrl in Controls) {
//     var oneJob              = ctrl.CreateJob();
//     oneJob.NumberOfMPIProcs = 1;
//     oneJob.Activate(myBatch); 
// }

In [ ]:
//wmg.AllJobs

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()